# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

In [1]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

## Basic FABRIC Slice Configuration

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
# Slice 
slice_name = 'P4Lang_Tutorial1'

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 40

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"

h1_ip="10.0.1.1"
h1_ip="10.0.2.2"
h1_ip="10.0.3.3"

host_cores = 2
host_ram = 8
host_disk = 10
# Sites
site_1 = 'MAX'
site_2 = 'TACC'
site_3 = 'UTAH'

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'



### Create the Slice

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site_1)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_image(image)
    [s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
    [s1_iface_to_s2, s1_iface_to_s3] = s1.add_component(model='NIC_ConnectX_5', name="s1_switch_nic").get_interfaces()
    s1_iface_to_s2.set_vlan(vlan=vlan)
    s1_iface_to_s3.set_vlan(vlan=vlan)

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site_2)
    s2.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2.set_image(image)
    [s2_iface_local] = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()
    [s2_iface_to_s1, s2_iface_to_s3] = s2.add_component(model='NIC_ConnectX_5', name="s2_switch_nic").get_interfaces()
    s2_iface_to_s1.set_vlan(vlan=vlan)
    s2_iface_to_s3.set_vlan(vlan=vlan)
   
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site_3)
    s3.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3.set_image(image)
    [s3_iface_local] = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()
    [s3_iface_to_s1, s3_iface_to_s2] = s3.add_component(model='NIC_ConnectX_5', name="s3_switch_nic").get_interfaces()
    s3_iface_to_s1.set_vlan(vlan=vlan)
    s3_iface_to_s2.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    [h1_iface] = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_2)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    [h2_iface] = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()
 
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site_3)
    h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h3.set_image(image)
    [h3_iface] = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])    
    
    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

Waiting for slice ............... Slice state: StableOK
Running post boot config ...Done!


## Get the Slice

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")  
            print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    for network in slice.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
   Cores             : 2
   RAM               : 8
   Disk              : 100
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : max-w4.fabric-testbed.net
   Site              : MAX
   Management IP     : 63.239.135.84
   Reservation ID    : 599df808-7ec7-4de1-9d92-d36533d51dc5
   Reservation State : Active
   SSH Command       : ssh -i /Users/pruth/.ssh/id_rsa -J pruth@bastion-1.fabric-testbed.net ubuntu@63.239.135.84
   Components        :  
      Name             : s1-s1_local_nic
      Details          : Mellanox ConnectX-6 VPI MCX653 dual port 100Gbps
      Disk (G)         : 0
      Units            : 1
      PCI Address      : 0000:e2:02.5
      Model            : ConnectX-6
      Type             : SharedNIC
      Name             : s1-s1_switch_nic
      Details          : Mellanox ConnectX-5 Dual Port 10/25GbE
      Disk (G)         : 0
      Units            : 1
      PCI Address      : ['0000:a1:00.0',

## Configure Nodes


In [ ]:
host_config_script='sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git'

try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=net_h1_name)  
    iface1.set_ip(ip=h1_ip, cidr="24")
    
    stdout, stderr = h1.execute(host_config_script)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.set_ip(ip=h2_ip, cidr="24")
    
    
    stdout, stderr = h2.execute(host_config_script)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
    
try:
    h3 = slice.get_node(name=h3_name)
    h3_os_iface = h2.get_interface(network_name=net_h3_name)
    h3_os_iface.set_ip(ip=h3_ip, cidr="24")
    
    stdout, stderr = h3.execute(host_config_script)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
    


## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



In [ ]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
    
    file_attributes = s1.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s1.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s1_h1_os_iface} {s1_s2_os_iface} {s1_s3_os_iface}  > /tmp/script.log 2>&1'")
    #print("stdout: {}".format(stdout))

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s1.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s2_s3_os_iface = s1.get_interface(network_name=net_s2_s3_name)
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s2.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s2_h2_os_iface} {s2_s1_os_iface} {s2_s3_os_iface}  > /tmp/script.log 2>&1'")
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    s3 = slice.get_node(name=s1_name)
    s3_h3_os_iface = s1.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s1.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s1.get_interface(network_name=net_s2_s3_name)
     
    file_attributes = s3.upload_file(scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s3.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s3_h3_os_iface} {s3_s1_os_iface} {s3_s2_os_iface}  > /tmp/script.log 2>&1'")
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [ ]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = experiment_topology.nodes[switch_name]
    management_ip_switch = str(switch_node.management_ip)
    print("Swtitch Name        : {}".format(switch_node.name))
    print("Management IP    : {}".format(management_ip_switch))
    
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = upload_file(username, switch_node, f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -it fabric_p4 simple_switch_CLI  > /tmp/script.log 2>&1'")
    stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

### Run the P4 Swtiches

In [ ]:
#for switch_name in [s1_name, s2_name, s3_name]:
for switch_name in [s1_name]:
    
    switch_node = experiment_topology.nodes[switch_name]
    management_ip_switch = str(switch_node.management_ip)
    print("Swtitch Name        : {}".format(switch_node.name))
    print("Management IP    : {}".format(management_ip_switch))

    iface_str = get_iface_list(switch_name)
    print(f"iface_str: {iface_str}")
    
    #file_attributes = upload_file(username, switch_node, 'router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {iface_str}  > /tmp/script.log 2>&1'")
    #print("stdout: {}".format(stdout))
    
    
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 cp solution/basic_tunnel.p4 basic_tunnel.p4'")
    #print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 p4c --p4runtime-files basic_tunnel.txt --target bmv2 --arch v1model basic_tunnel.p4'")
    #print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -d -w /root/tutorials/exercises/basic_tunnel fabric_p4 simple_switch --interface 0@ens7 --interface 1@ens8 --interface 2@ens9 basic_tunnel.json'")
    #print("stdout: {}".format(stdout))
    
    
    #echo "echo This is how we pipe to docker exec" | sudo docker exec --interactive CONTAINER_NAME /bin/bash - 


    #stdout = execute_script(username, switch_node, f"sudo sh -c 'echo table_set_default myTunnel_exact drop | docker exec -i -w /root/tutorials/exercises/basic_tunnel fabric_p4 /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    #print("stdout: {}".format(stdout)) 
    #stdout = execute_script(username, switch_node, f'echo "table_add myTunnel_exact myTunnel_forward 1 \=\> 1" | sudo sh -c docker exec -i -w /root/tutorials/exercises/basic_tunnel fabric_p4 /usr/local/bin/simple_switch_CLI --thrift-port 9090')
    #print("stdout: {}".format(stdout))
    stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -i -w /root/tutorials/exercises/basic_tunnel fabric_p4 echo \"table_add myTunnel_exact myTunnel_forward 1 => 1\" \\| /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 echo \'table_add myTunnel_exact myTunnel_forward 2 => 2\' \| /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    #print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 echo \'table_add myTunnel_exact myTunnel_forward 3 => 3\' \| /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    #print("stdout: {}".format(stdout))

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()